# KAFKA PRODUCER

In [5]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
import tweepy
import os
import json
from kafka import KafkaProducer

ACCESS_TOKEN=os.getenv('ACCESS_TOKEN')
ACCESS_TOKEN_SECRET=os.getenv('ACCESS_TOKEN_SECRET')
CONSUMER_KEY=os.getenv('CONSUMER_KEY')
CONSUMER_SECRET=os.getenv('CONSUMER_SECRET')

#SEARCH_KEYWORD = str(input("Please enter a keyword to filter\n"))

producer = KafkaProducer(bootstrap_servers=['kafka:9092'])

def filter_tweet(tweet):
    data = json.loads(tweet)
    
    if 'delete' in data:
        return False
    if 'timestamp_ms' not in data or 'entities' not in data:
        return False      
    if len(data['entities']['hashtags']) <= 0:
        return False
        
    data = {"timestamp": data['timestamp_ms'], "hashtags": [i["text"] for i in data['entities']['hashtags']]}
    print(data)
    data = json.dumps(data)
    
    return data


class StdOutListener(StreamListener):
    def __init__(self):
        self.count=0

    def on_data(self, data):
        data = filter_tweet(data)
        if data != False:
            producer.send("tweets", data.encode('utf-8'))  
         
        self.count+=1 
        print(self.count)
        return True
    def on_error(self, status):
        print(status)
        
    def on_exception(self, exception):
        print(exception)
        return

myStreamListener = StdOutListener()
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
myStream = tweepy.Stream(auth = auth, listener=myStreamListener)
#myStream.filter(track=[SEARCH_KEYWORD])
myStream.filter(languages=["en"], track=["a", "the", "i", "you", "u"])
#myStream.sample(languages=["en"])

print(f"Starting tweet streams")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
{'timestamp': '1624627792798', 'hashtags': ['YUGYEOM']}
31
32
33
{'timestamp': '1624627792804', 'hashtags': ['MakeItSmart']}
34
35
36
37
38
39
40
{'timestamp': '1624627792791', 'hashtags': ['camren']}
41
42
43
44
45
{'timestamp': '1624627792829', 'hashtags': ['AESPAdiTokopediaWIB', 'WIBTVSHOWJuni']}
46
{'timestamp': '1624627792805', 'hashtags': ['Mandalay', 'WhatsHappeningInMyanmar', 'June25Coup', 'MilkTeaAlliance']}
47
48
{'timestamp': '1624627792819', 'hashtags': ['coway', 'CowayBTS', 'bts', 'btsarmy', '방탄소년단']}
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
{'timestamp': '1624627793671', 'hashtags': ['DiorBoySEHUN']}
79
{'timestamp': '1624627793649', 'hashtags': ['เคอแพท', 'Kepat', 'KepatFanart', 'PP6D']}
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
{'timestamp': '1624627793695', 'hashtags': ['ElectricVehicle']}
101
102
103
104
105
106
107
10

KeyboardInterrupt: 